In [ ]:
import os, csv
from dotenv import load_dotenv
import pandas as pd
import time

from datetime import datetime
import schedule
import requests

import calendar
import pandas as pd
import pandas_ta as ta
import numpy as np

load_dotenv()

### Set Environment

In [ ]:
GLOBLEX_ID = os.getenv('GLOBLEX_ID')
SETTRADE_APP_ID = os.getenv('SETTRADE_APP_ID')
SETTRADE_APP_SECRET = os.getenv('SETTRADE_APP_SECRET')
# GLOBLEX_ID, SETTRADE_APP_ID, SETTRADE_APP_SECRET

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

### Prepare Data

In [ ]:
def get_initial_bars(asset, tf): # params: asset = asset name, tf = timeframe such as 5m, 15m, 30m, 1h, 4h.
    df = {}
    # TODO
    # download latest 200 or more bars.
    # convert columns to open, high, low, close.
    # Sort ascending datetime, latest is in last row.
    # set index by datetime.
    # return dataframe

    """ example code
    bars, _ = get_from_globex()
    df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with strategy

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.iloc[::-1] # reverse row
    df.reset_index(drop=True, inplace=True)
    """

    return df

In [2]:
def get_latest_bars(df, asset, tf): # params: df=dataframe, asset=asset name, tf=timeframe such as 5m, 15m, 30m, 1h, 4h.
    # TODO
    # get latest 1 bar.
    # convert columns to match with dataframe
    # set index by datetime
    # merge latest 1 bars to dataframe
    # return dataframe

    """ example code
    bars, _ = get_from_globex()
    incoming_df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with rsi strategy

    incoming_df['timestamp'] = pd.to_datetime(incoming_df['timestamp'], unit='ms')
     
    if (incoming_df['timestamp'].iloc[0] != df['timestamp'].iloc[-1]):
        df = df[:900].append(incoming_df.head(1), ignore_index = True)
    """
    return df

In [3]:
def notifyMsg(*msg): # notify line message
    r = requests.post(LINE_URL, headers=LINE_HEADERS, data={'message': msg[0]})
    print(r.text)

### Strategy

In [ ]:
class TFEX_Bot:
    def __init__(self, params):
        self.client = None
        self.asset = params['asset']
        
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.current_action = 'close'
        
        # TODO: create params for indicator
        self.ema = params['ema']
        self.rr = params['rr']
        ###-------------------###
        
        self.lot_size = 0.01
        self.order_id = 0
        self.target = 10
        self.round = params['round']
        
        self.df = None
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def calLotSize(self, entry, exit):
        # TODO: calculate lot size
        # self.lot_size = abs(round(self.target / (((entry - exit) / self.pip ) * self.pip_value ) , 2))
        # print(f'lot_size: {str(self.lot_size)}')
        
    def executeOrder(self, action, entry=0, exit=0, sl=0, tp=0):
        # params: action=buy or sell, entry=entry price, exit=exit price, sl=stop loss, tp= take profit

        # TODO
        # create globex conection
        # login globex and check conection
        # if connection ok then execute order
        # get successfully excute order response and notify to Line
        # if order execution is success then set state to buy
        # disconnect

        """ example
        self.client = GlobexClient(port=9999)

        loginResponse = self.client.execute(loginCommand(userId=GLOBLEX_ID, password=))
        logger.info(str(loginResponse)) 

        # check if user logged in correctly
        if(loginResponse['status'] == False):
            print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))
            
        self.client.disconnect()
        """
        
    def do_strategy(self, bars): # bars=price dataframe
        self.df = bars
        strategy = ta.Strategy(
            name = 'TFEX bot',
            description = 'TFEX bot',
            ta = [
                {'kind': 'ema', 'length': self.ema}
            ]
        )
        self.df.ta.strategy(strategy)
    
        row = self.df.tail(1).to_dict('records')[0] # latest bar
        
        if self.current_action == 'close' or self.order == {}: # wait for buy signal
            if (row['EMA_'+str(self.ema)] < row['low']):
                self.order['action'] = 'buy'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr)]
                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'buy'
                notifyMsg(f"Buy {self.pair} TFEX\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                
                self.calLotSize(self.order['open'], self.order['T/P'])
                self.executeOrder('buy', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])

        if (self.current_action == 'buy'): # wait for TP or SL
            if (row['low'] <= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                notifyMsg(f"S/L Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.executeOrder('sell', entry=self.order['open'], exit=self.order['T/P'], sl=self.order['S/L'], tp=self.order['T/P'])
                self.reset_order()
                
            elif (row['high'] >= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                notifyMsg(f"T/P Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()


### Run Bot

In [ ]:
tfex_bot = TFEX_Bot({
    'pair': 'TFEX',
    'ema': 100,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'round': 5,
})

In [ ]:
tfex_init_bars = get_initial_bars('TFEX', tf='15min')

In [ ]:
def get_realtime():
    tfex_bars = get_latest_bars(tfex_init_bars, pair='TFEX', tf='15min')
    tfex_bot.do_strategy(tfex_bars)
    
schedule.every(60 * 15).seconds.do(get_realtime) # every 15min

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)